## Задание 1

In [28]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston()
data = boston['data']

feature_names = boston['feature_names']
X = pd.DataFrame(data, columns=feature_names)
# X
# boston.keys()
# boston['target']

y = pd.DataFrame(boston['target'], columns=['price'])
# y

In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
# y_test

In [52]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
# type(lr)
lr.fit(X_train, y_train)

LinearRegression()

In [72]:
y_predict = lr.predict(X_test)
# y_predict
system_check = pd.DataFrame({
    'y_test' : y_test['price'],
    'y_pred' : y_predict.flatten()
})

In [55]:
from sklearn.metrics import r2_score

r2 = r2_score(system_check['y_test'], system_check['y_pred'])
r2

0.7112260057484912

## Задание 2

In [78]:
import warnings

warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 1000, max_depth = 12, random_state = 42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
model.fit(X_train, y_train.values[:,0])

y_predict = model.predict(X_test)

# system_check = pd.DataFrame({
#     'y_test' : y_test['price'],
#     'y_pred' : y_predict
# })

model.score(X_test, y_test)

0.87472606157312

##### Таким образом, случайный лес демонстрирует более высокий R2, чем линейная модель, то есть объясняет большую долю дисперсии

## Задание 3

In [79]:
?RandomForestRegressor

In [108]:
predictor_values = model.feature_importances_

sum_predictors = np.sum(predictor_values)

1.0

##### сумма всех предикторов:

In [109]:
sum_predictors

1.0

##### самые важные показатели - LSTAT и RM

In [110]:
predictors = pd.DataFrame(data = {'values' : predictor_values}, index = model.feature_names_in_)
predictors.sort_values(by='values', ascending = False).head(2)

,values
LSTAT,0.415847
RM,0.402682


## Задание 4